In [19]:
import os
import random
import string
import hashlib
import datetime
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Sequence
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from flask import Flask, jsonify, send_from_directory
import tkinter as tk
from tkinter import ttk, messagebox
import tensorflow as tf
import numpy as np
from pyngrok import ngrok
import logging

# Global variables
seq_length = 128
input_dim = 1
model = None  # AI model for smart fuzzing
logging.basicConfig(level=logging.DEBUG)

# Database Setup
DATABASE_URL = "sqlite:///vulnerabilities.db"  # You can change this to your preferred database
engine = create_engine(DATABASE_URL)
Base = declarative_base()

# Vulnerability Log Model
class VulnerabilityLog(Base):
    __tablename__ = 'vulnerability_logs'
    id = Column(Integer, Sequence('vuln_id_seq'), primary_key=True)
    timestamp = Column(DateTime, default=datetime.datetime.utcnow)
    vulnerability = Column(String(255))
    hash = Column(String(64))

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Function to Hash Data
def hash_data(data):
    return hashlib.sha256(data.encode()).hexdigest()

# Function to Log Vulnerability
def log_vulnerability(vuln_data):
    vuln_hash = hash_data(vuln_data)
    vulnerability = VulnerabilityLog(vulnerability=vuln_data, hash=vuln_hash)
    session.add(vulnerability)
    session.commit()
    print(f"Logged vulnerability: {vuln_data} with hash: {vuln_hash}")

# Fuzzing Functions
def fuzz_input():
    sensor_data = ''.join(random.choices(string.printable, k=128))
    log_vulnerability("Fuzzed sensor data vulnerability")

def fuzz_protocol():
    command = ''.join(random.choices(string.printable, k=128))
    log_vulnerability("Fuzzed command protocol vulnerability")

def fuzz_target_info():
    target_info = ''.join(random.choices(string.printable, k=128))
    log_vulnerability("Fuzzed target info vulnerability")

# AI-Powered Smart Fuzzing
def generate_smart_fuzzing_inputs(model, size, batch_size=1):
    fuzzed_data = model.predict(np.random.rand(batch_size, size, input_dim))
    return fuzzed_data

# Flask App for Visualization
app = Flask(__name__)

# Serve static files (index.html, CSS, JS)
@app.route('/')
def serve_index():
    return send_from_directory('static', 'index.html')

@app.route('/static/<path:filename>')
def serve_static(filename):
    return send_from_directory('static', filename)

# API to fetch vulnerabilities
@app.route('/vulnerabilities')
def get_vulnerabilities():
    vulnerabilities = session.query(VulnerabilityLog).all()
    vuln_list = [{"id": v.id, "timestamp": v.timestamp, "vulnerability": v.vulnerability, "hash": v.hash} for v in vulnerabilities]
    return jsonify(vuln_list)

# Create LSTM Model
def create_lstm_model(input_dim, seq_length):
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(128, input_shape=(seq_length, input_dim), return_sequences=True),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(input_dim, activation='sigmoid')  # Output should be in the range of your input data
    ])
    model.compile(optimizer='adam', loss='mse')  # Or another appropriate loss function
    return model

# Tkinter GUI Setup
def setup_gui():
    root = tk.Tk()
    root.title("🛡 AI-Powered Vulnerability Logger")
    root.geometry("1000x400")

    # Create a frame for the table
    frame = ttk.Frame(root)
    frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

    # Create a Treeview widget for the table
    columns = ("Timestamp", "Vulnerability", "Hash")
    tree = ttk.Treeview(frame, columns=columns, show="headings")
    tree.heading("Timestamp", text="Timestamp")
    tree.heading("Vulnerability", text="Vulnerability")
    tree.heading("Hash", text="Hash")
    tree.column("Timestamp", width=150, anchor="center")
    tree.column("Vulnerability", width=600, anchor="w")
    tree.column("Hash", width=200, anchor="w")
    tree.pack(fill=tk.BOTH, expand=True)

    # Add a scrollbar
    scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=tree.yview)
    tree.configure(yscroll=scrollbar.set)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    # Add buttons
    btn_frame = ttk.Frame(root)
    btn_frame.pack(pady=10)

    btn_fuzz = ttk.Button(btn_frame, text="🔍 Run Fuzzing", command=lambda: start_fuzzing(tree))
    btn_fuzz.grid(row=0, column=0, padx=10, pady=5)

    btn_clear = ttk.Button(btn_frame, text="🗑 Clear Logs", command=lambda: clear_logs(tree))
    btn_clear.grid(row=0, column=1, padx=10, pady=5)

    btn_refresh = ttk.Button(btn_frame, text="🔄 Refresh Table", command=lambda: refresh_table(tree))
    btn_refresh.grid(row=0, column=2, padx=10, pady=5)

    return root

# Function to start fuzzing
def start_fuzzing(tree):
    global model
    try:
        # Perform fuzzing operations
        for _ in range(1):  # Run fuzzing 5 times
            fuzz_input()
            fuzz_protocol()
            fuzz_target_info()

        # AI Smart Fuzzing
        if model is not None:
            fuzzing_inputs = generate_smart_fuzzing_inputs(model, seq_length, batch_size=1)
            print(f"AI-Powered Fuzzing Inputs: {fuzzing_inputs}")
    except Exception as e:
        print(f"Error in fuzzing loop: {e}")
    finally:
        refresh_table(tree)

# Function to refresh the table
def refresh_table(tree):
    tree.delete(*tree.get_children())  # Clear the table
    vulnerabilities = session.query(VulnerabilityLog).all()
    for vuln in vulnerabilities:
        tree.insert("", "end", values=(vuln.timestamp, vuln.vulnerability, vuln.hash))

# Function to clear logs
def clear_logs(tree):
    session.query(VulnerabilityLog).delete()
    session.commit()
    refresh_table(tree)
    print("All logs cleared.")

# Main Function
def main():
    global model
    try:
        # Load or create AI Model
        model = create_lstm_model(input_dim, seq_length)
        print("Model loaded/created successfully.")
    except Exception as e:
        print(f"Error loading/creating model: {e}")

    # Start the Tkinter GUI
    root = setup_gui()
    root.mainloop()

# Run Main Function
if __name__ == "__main__":
    main()

C:\Users\Nidhi K N\AppData\Local\Temp\ipykernel_15280\175781493.py:26: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


Model loaded/created successfully.
Logged vulnerability: Fuzzed sensor data vulnerability with hash: 21d148b1a82c8b3532051f3dcd1727360a29b191430e6aefbd4a8b1e39e31260
Logged vulnerability: Fuzzed command protocol vulnerability with hash: 51cfeb03273dd3ac6a944ce66f75f7bbc2de61c107ebbada7648fa05d2ee92a4
Logged vulnerability: Fuzzed target info vulnerability with hash: b924fb2d80ad259348da2207ac993dec001fa07c433ea4e3ca23d68f6bd5f11b
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
AI-Powered Fuzzing Inputs: [[0.49571458]]
